# Checar antes de tramitar

In [1]:
import os
import sys
import math
import locale
import pypdf
sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np

from typing import List
from pydantic import BaseModel, Field
from utils_ccd import get_connection, get_info_file_path, extract_text_from_pdf, generate_pdf_office
from datetime import datetime
from dotenv import load_dotenv
from docxtpl import DocxTemplate

load_dotenv()
conn = get_connection()
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

from pathlib import Path
from langchain_openai import AzureOpenAI, AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, FewShotChatMessagePromptTemplate
from docx2pdf import convert

c:\Users\05911205424\Documents\Dev\ccd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sql_nereu_gccth = '''  
SELECT 
	CONCAT(p.numero_processo, '/', p.ano_processo) as processo,
	pm.Descricao as marcador
FROM processo.dbo.Processos p
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON p.IdProcesso = pmp.IdProcesso 
INNER JOIN processo.dbo.Pro_Marcador pm ON pm.IdMarcador = pmp.IdMarcador 
INNER JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = p.IdProcesso 
INNER JOIN processo.dbo.Processos pori ON ed.IdProcessoOrigem = pori.IdProcesso 
WHERE p.setor_atual = 'CCD'
AND pm.Descricao LIKE '%CCTH%'
'''
df_gccth = pd.read_sql(sql_nereu_gccth, conn)

In [3]:
df_gccth

,processo,marcador
0,000115/2023,GCCTH - SUSPENSÃO NEREU
1,000116/2023,GCCTH - SUSPENSÃO NEREU
2,000114/2023,GCCTH - SUSPENSÃO NEREU
3,000117/2023,GCCTH - SUSPENSÃO NEREU
4,003065/2022,GCCTH - SUSPENSÃO NEREU
...,...,...
76,003696/2022,GCCTH - SUSPENSÃO NEREU
77,003075/2022,GCCTH - SUSPENSÃO NEREU
78,003693/2022,GCCTH - SUSPENSÃO NEREU
79,003694/2022,GCCTH - SUSPENSÃO NEREU


In [4]:
sql_infos_gccth = '''  

SELECT CONCAT(vai.numero_processo, '/', vai.ano_processo) as processo
FROM vw_ata_informacao vai 
WHERE CONCAT(vai.numero_processo, '/', vai.ano_processo) IN (
SELECT 
	CONCAT(p.numero_processo, '/', p.ano_processo) as processo
FROM processo.dbo.Processos p
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp ON p.IdProcesso = pmp.IdProcesso 
INNER JOIN processo.dbo.Pro_Marcador pm ON pm.IdMarcador = pmp.IdMarcador 
INNER JOIN processo.dbo.Exe_Debito ed ON ed.IdProcessoExecucao = p.IdProcesso 
INNER JOIN processo.dbo.Processos pori ON ed.IdProcessoOrigem = pori.IdProcesso 
WHERE p.setor_atual = 'CCD'
AND pm.Descricao LIKE '%CCTH%'
)
AND vai.setor = 'CCD'
AND resumo LIKE '%sobrest%'
'''
df_infos_gccth = pd.read_sql(sql_infos_gccth, conn)

In [8]:
set(df_infos_gccth['processo']) - set(df_gccth['processo'])

set()